Import necessary libraries and define the functions

In [30]:
import pandas as pd
import glob
import os
import sqlite3

def read_and_process_excel(file_pattern, columns_to_keep, extract_item_type=False):
    """Reads and processes Excel files matching a pattern, with optional extraction of item type."""
    xlsx_files = glob.glob(file_pattern)
    all_data = []

    for file in xlsx_files:
        if 'predictions' in file.lower():
            print(f"\nProcessing file: {file}")
            try:
                df = pd.read_excel(file, engine='openpyxl')
                print(f"Successfully read {file}.")
            except Exception as e:
                print(f"Error reading {file}: {e}")
                continue

            # Extract item type and location based on the file type
            base_name = os.path.basename(file).replace('.xlsx', '').replace('predictions', '').replace('2024', '').strip()

            if extract_item_type:
                # For specific product predictions, split item type and location
                item_type = ''.join(filter(str.isalpha, base_name.split('Bayern')[0].split('Brandenburg')[0]))  # ItemType part
                location_name = base_name.replace(item_type, '').strip()  # Location part
                df['ItemType'] = item_type
                df['Location'] = location_name
            else:
                # For general predictions, location is the only part
                df['Location'] = base_name

            # Keep only relevant columns
            df = df[columns_to_keep]

            # Ensure numeric columns are floats and clean NaNs
            numeric_cols = ['Quantity', 'Train', 'Test', 'Predict']
            df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

            # Remove Quantity values where Quantity == Predict
            df.loc[df['Predict'] == df['Quantity'], 'Quantity'] = pd.NA

            # Handle NaNs and rounding
            df[numeric_cols] = df[numeric_cols].fillna(0).round(2).replace(0, pd.NA)

            # Add to data list
            all_data.append(df)
        else:
            print(f"Skipping file: {file} (does not contain 'predictions')")

    # Combine all DataFrames
    combined_df = pd.concat(all_data, ignore_index=True)
    print("\nCombined DataFrame (first few rows):")
    print(combined_df.head())
    return combined_df

def save_to_sqlite(df, db_name, table_name):
    """Saves the DataFrame to SQLite database."""
    with sqlite3.connect(db_name) as conn:
        conn.execute(f"DROP TABLE IF EXISTS {table_name}")
        df.to_sql(table_name, conn, if_exists='append', index=False)
        df_all = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5", conn)
        print(f"\nPreview of table '{table_name}' after saving:")
        print(df_all)







Successfully processed: data/LSTM/Items/New\2024predictionsBaden-Württemberg.xlsx

Successfully processed: data/LSTM/Items/New\2024predictionsBayern.xlsx

Successfully processed: data/LSTM/Items/New\2024predictionsBrandenburg.xlsx

Successfully processed: data/LSTM/Items/New\2024predictionsNordrhein-Westfalen.xlsx

Combined DataFrame (first few rows):
        Date Quantity Train  Test Predict           Location
0 2021-12-27   5445.0  <NA>  <NA>    <NA>  Baden-Württemberg
1 2022-01-03  22495.0  <NA>  <NA>    <NA>  Baden-Württemberg
2 2022-01-10  42228.0  <NA>  <NA>    <NA>  Baden-Württemberg
3 2022-01-17  38619.0  <NA>  <NA>    <NA>  Baden-Württemberg
4 2022-01-24  43536.0  <NA>  <NA>    <NA>  Baden-Württemberg

Preview of table 'predictions_LSTM_Items' after saving:
                  Date Quantity Train  Test Predict           Location
0  2021-12-27 00:00:00   5445.0  None  None    None  Baden-Württemberg
1  2022-01-03 00:00:00  22495.0  None  None    None  Baden-Württemberg
2  2022-0

Process general predictions of Items

In [ ]:
columns_to_keep_general = ['Date', 'Quantity', 'Train', 'Test', 'Predict', 'Location']
combined_df_general = read_and_process_excel('data/LSTM/Items/New/*.xlsx', columns_to_keep_general, extract_item_type=False)

# Save general predictions to SQLite
save_to_sqlite(combined_df_general, 'predictions_LSTM.db', 'predictions_LSTM_Items')


Process specific product subgroup predictions

In [ ]:
columns_to_keep_products = ['Date', 'Quantity', 'Train', 'Test', 'Predict', 'ItemType', 'Location']
combined_df_products = read_and_process_excel('data/LSTM/Items/ItemPredicting/*.xlsx', columns_to_keep_products, extract_item_type=True)

# Save specific product predictions to SQLite
save_to_sqlite(combined_df_products, 'predictions_LSTM.db', 'predictions_LSTM_Items_SpecificProducts')

Load CSV for item clustering and save to SQLite

In [ ]:

csv_file = 'data/LSTM/Items/itemClustering.csv'
df_clustering = pd.read_csv(csv_file, sep=';', header=0)
print("\nClustering Data (first few rows):")
print(df_clustering.head())

# Save clustering data to SQLite
save_to_sqlite(df_clustering, 'predictions_LSTM.db', 'predictions_Item_Clustering')